In [1]:
import pickle
from datetime import datetime
from dateutil.tz import tzoffset

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from torch import nn
from torch import optim
from scipy.stats import kendalltau, spearmanr
from sklearn.preprocessing import OneHotEncoder

# Задача 1

Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
- взять в тренировочный набор турниры с dateStart из 2019 года; 
- в тестовый — турниры с dateStart из 2020 года.

In [2]:
with open("players.pkl", "rb") as f:
    players = pickle.load(f)

In [3]:
with open("results.pkl", "rb") as f:
    results = pickle.load(f)

In [4]:
with open("tournaments.pkl", "rb") as f:
    tournaments = pickle.load(f)

In [5]:
dateformat = "%Y-%m-%dT%H:%M:%S%z"

In [6]:
def get_data(y, test=False):
    data = []
    for key, value in results.items():
        for team in value:
            for team_member in team["teamMembers"]:
                tournament_year = datetime.strptime(tournaments[key]["dateStart"], dateformat).year
                if team.get("mask", None) and tournament_year == y:
                    dct = {
                        "tournament_id": key,
                        "tournament_name": tournaments.get(key).get("name"),
                        "team_id": team["team"]["id"],
                        "team_name": team["team"]["name"],
                        "questions_mask": team.get("mask", None),
                        "questionQty": tournaments.get(key).get("questionQty"),
                        "position": team.get("position", None),
                        "player_id": team_member["player"]["id"],
                        "player_name": team_member["player"]["surname"] + " " + \
                                       team_member["player"]["name"] + " " + \
                                       team_member["player"]["patronymic"]
                    }
                    if test:
                        dct.pop("questions_mask", None)
                        dct.pop("questionQty", None)
                    data.append(dct)
    return data

In [7]:
train_data = pd.DataFrame(get_data(2019))
train_data["questionQty"] = train_data["questionQty"].apply(lambda x: sum(list(x.values())))
train_data = train_data[train_data["questions_mask"].apply(len) == train_data["questionQty"]].copy()
test_data = pd.DataFrame(get_data(2020, True))

In [8]:
train_data

,tournament_id,tournament_name,team_id,team_name,questions_mask,questionQty,position,player_id,player_name
0,4772,Синхрон северных стран. Зимний выпуск,45556,Рабочее название,111111111011111110111111111100010010,36,1.0,6212,Выменец Юрий Яковлевич
1,4772,Синхрон северных стран. Зимний выпуск,45556,Рабочее название,111111111011111110111111111100010010,36,1.0,18332,Либер Александр Витальевич
2,4772,Синхрон северных стран. Зимний выпуск,45556,Рабочее название,111111111011111110111111111100010010,36,1.0,18036,Левандовский Михаил Ильич
3,4772,Синхрон северных стран. Зимний выпуск,45556,Рабочее название,111111111011111110111111111100010010,36,1.0,22799,Николенко Сергей Игоревич
4,4772,Синхрон северных стран. Зимний выпуск,45556,Рабочее название,111111111011111110111111111100010010,36,1.0,15456,Коновалов Сергей Владимирович
...,...,...,...,...,...,...,...,...,...
450511,6255,ОВСЧ,60897,Тесла на колёсах,0010000000000110000000001000000000000010000000...,216,1730.0,201802,Тренина Анастасия Андреевна
450512,6255,ОВСЧ,60897,Тесла на колёсах,0010000000000110000000001000000000000010000000...,216,1730.0,207830,Давлетбаков Елисей Сергеевич
450513,6255,ОВСЧ,60897,Тесла на колёсах,0010000000000110000000001000000000000010000000...,216,1730.0,207831,Козлов Иван Витальевич
450514,6255,ОВСЧ,60897,Тесла на колёсах,0010000000000110000000001000000000000010000000...,216,1730.0,207832,Орлов Дмитрий Алексеевич


In [9]:
test_data

,tournament_id,tournament_name,team_id,team_name,position,player_id,player_name
0,4957,Синхрон Биркиркары,49804,Борский корабел,1.0,30152,Сорожкин Артём Сергеевич
1,4957,Синхрон Биркиркары,49804,Борский корабел,1.0,30270,Спешков Сергей Леонидович
2,4957,Синхрон Биркиркары,49804,Борский корабел,1.0,27822,Савченков Михаил Владимирович
3,4957,Синхрон Биркиркары,49804,Борский корабел,1.0,28751,Семушин Иван Николаевич
4,4957,Синхрон Биркиркары,49804,Борский корабел,1.0,27403,Руссо Максим Михайлович
...,...,...,...,...,...,...,...
112836,6456,Онлайн: 16:00 Зелёный шум,69918,Teddyhaters,6.0,129706,Трилис Алексей Андреевич
112837,6456,Онлайн: 16:00 Зелёный шум,69918,Teddyhaters,6.0,192901,Федоркина Мария Олеговна
112838,6456,Онлайн: 16:00 Зелёный шум,63129,Якуба,7.0,165962,Горбиль Павел Романович
112839,6456,Онлайн: 16:00 Зелёный шум,63129,Якуба,7.0,154624,Пильненький Артём Сергеевич


# Задача 2

Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков.

- повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
- в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
- для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.

In [10]:
question_answer_data = {
    "question": [],
    "player": [],
    "initial_label": [],
    "team_id": [],
    "tournament_id": []
}
for tour_id, team_id, player_id, mask in zip(train_data["tournament_id"], train_data["team_id"], train_data["player_id"], train_data["questions_mask"]):
    for i, result in enumerate(mask):
        if result != "X" and result != "?":
            question_answer_data["tournament_id"].append(tour_id)
            question_answer_data["team_id"].append(team_id)
            question_answer_data["question"].append(f"{tour_id}_{i}")
            question_answer_data["player"].append(player_id)
            question_answer_data["initial_label"].append(int(result))

In [11]:
question_answer_data = pd.DataFrame(question_answer_data)
question_answer_data

,question,player,initial_label,team_id,tournament_id
0,4772_0,6212,1,45556,4772
1,4772_1,6212,1,45556,4772
2,4772_2,6212,1,45556,4772
3,4772_3,6212,1,45556,4772
4,4772_4,6212,1,45556,4772
...,...,...,...,...,...
17739653,6255_211,210786,0,60897,6255
17739654,6255_212,210786,0,60897,6255
17739655,6255_213,210786,0,60897,6255
17739656,6255_214,210786,0,60897,6255


In [12]:
class LogisticRegression(nn.Module):
    def __init__(self, n_features):  
        super().__init__()
        self.fc = nn.Linear(n_features, 1)
        self.sigmoid = nn.Sigmoid()     
    def forward(self, x):
        return self.sigmoid(self.fc(x))

In [13]:
def m_step(model, x, y, lr=1, n_iter=250):
    model.fc.reset_parameters()
    criteria = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    for i in range(n_iter):
        optimizer.zero_grad()
        output = model(x)
        loss = criteria(output, y)
        loss.backward()
        optimizer.step()

In [14]:
encoder = OneHotEncoder()

train_data = encoder.fit_transform(question_answer_data[["player", "question"]])
train_data

<17739658x90728 sparse matrix of type '<class 'numpy.float64'>'
	with 35479316 stored elements in Compressed Sparse Row format>

In [15]:
def get_top_players(parameters, n=15):
    player_weights = {}
    for i, c in enumerate(encoder.get_feature_names()):
        if c.startswith("x0_"):
            player_weights[int(c[3:])] = parameters[i]
    top_players = sorted([(k, v) for k, v in player_weights.items()], reverse=True, key=lambda x: x[1])[:n]
    top_players = [" ".join([players[i]["surname"], players[i]["name"],players[i]["patronymic"]]) for i, e in top_players]
    return pd.DataFrame(top_players)

In [16]:
train_data = train_data.tocoo()

x = torch.sparse.FloatTensor(
    torch.LongTensor(np.vstack((train_data.row, train_data.col))),
    torch.FloatTensor(train_data.data)
)

y = torch.FloatTensor(question_answer_data["initial_label"].values).view(-1, 1)

model = LogisticRegression(x.shape[1])

In [17]:
%%time
m_step(model, x, y)

Wall time: 6min 55s


# Задача 3

Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
- предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
- в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.


In [18]:
def calc_metrics(data, encoder, parameters, intercept):
    player_weights = {}
    count = 0
    question_sum = 0
    player_sum = 0
    player_cnt = 0
    for i, c in enumerate(encoder.get_feature_names()):
        if c.startswith("x0_"):
            player_weights[int(c[3:])] = parameters[i]
            player_sum += parameters[i]
            player_cnt += 1
        else:
            question_sum += parameters[i]
            count += 1
    question_mean = question_sum / count
    data["player_weights"] = data["player_id"].map(player_weights)
    data["player_weights"].fillna(player_sum / player_cnt, inplace=True)
    data["players_proba"] = data["player_weights"].apply(lambda x: 1 / (1 + np.exp(-(x + question_mean + intercept))))
    probas = data.groupby(["tournament_id", "team_id"])["players_proba"].apply(lambda x: np.prod(1 - x))
    position = data.groupby(["tournament_id", "team_id"])["position"].first()
    group_data = pd.concat([probas, position], axis=1)
    group_data.sort_values(["tournament_id", "players_proba"], ascending=[True, True], inplace=True)
    spear = group_data.groupby("tournament_id").apply(lambda x: spearmanr(x["position"], x["players_proba"]).correlation).mean()
    kendl = group_data.groupby("tournament_id").apply(lambda x: kendalltau(x["position"], x["players_proba"]).correlation).mean()
    print(f"spearman: {spear}")
    print(f"kendl: {kendl}")

In [19]:
calc_metrics(test_data, encoder, model.fc.weight.data[0].numpy(), model.fc.bias.data[0].numpy())

spearman: 0.7860756536377214
kendl: 0.6292057479880974


In [20]:
get_top_players(model.fc.weight.data[0].numpy())

,0
0,Руссо Максим Михайлович
1,Брутер Александра Владимировна
2,Семушин Иван Николаевич
3,Савченков Михаил Владимирович
4,Кудинов Дмитрий Сергеевич
5,Спешков Сергей Леонидович
6,Сорожкин Артём Сергеевич
7,Пилипенко Максим Игоревич
8,Мереминский Станислав Григорьевич
9,Подюкова Валентина


# Задача 4

Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
- предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
- разработайте EM-схему для обучения этой модели, реализуйте её в коде;
- обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.


In [21]:
def e_step(data, predicts):
    data["label"] = predicts
    data.loc[data["initial_label"] == 0, "label"] = 0
    idx = data["initial_label"] == 1
    sp = data.loc[idx].groupby(["team_id", "question"])["label"].transform(lambda x: 1 - np.prod(1 - x.values))
    data.loc[idx, "label"] = data.loc[idx, "label"] / sp  
    return data

In [22]:
predicts = model(x).detach().numpy().ravel()
for i in range(5):
    question_answer_data = e_step(question_answer_data, predicts)
    y = torch.FloatTensor(question_answer_data["label"].values).view(-1, 1)
    m_step(model, x, y)
    torch.save(model.state_dict(), f"model_{i}.pth")
    predicts = model(x).detach().numpy().ravel()
    print(f"Iter: {i}")
    calc_metrics(test_data, encoder, model.fc.weight.data[0].numpy(), model.fc.bias.data[0].numpy())

Iter: 0
spearman: 0.7937743602872681
kendl: 0.6381819769337901
Iter: 1
spearman: 0.7935780095779675
kendl: 0.6378959752536173
Iter: 2
spearman: 0.7959434066855882
kendl: 0.6400247503161611
Iter: 3
spearman: 0.7961826065405053
kendl: 0.6402572457370044
Iter: 4
spearman: 0.7987802365065366
kendl: 0.6432934951232628


# Задача 5

А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

In [23]:
parameters = model.fc.weight.data[0].numpy()

question_weights = {}
for i, c in enumerate(encoder.get_feature_names()):
    if c.startswith("x1_"):
        question_weights[c[3:]] = parameters[i]

In [24]:
weights = question_answer_data.groupby("tournament_id")["question"].apply(lambda x: np.mean([question_weights[q] for q in x])).sort_values()

In [25]:
qw = weights.reset_index()["tournament_id"].apply(lambda x: tournaments[x]["name"])

In [26]:
qw.head()

0    Чемпионат Санкт-Петербурга. Первая лига
1                     Чемпионат Таджикистана
2     Зеркало мемориала памяти Михаила Басса
3                                Угрюмый Ёрш
4                         Воображаемый музей
Name: tournament_id, dtype: object

In [27]:
qw.tail()

670                           Школьная лига. III тур.
671    Студенческий чемпионат Калининградской области
672                               Синхрон Лиги Разума
673                 Шестой киевский марафон. Асинхрон
674                            Асинхрон по South Park
Name: tournament_id, dtype: object